In [46]:
import sys

import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.analysis.make_analysis_data as ad
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def user_period_data(df, user_id, period):
    return (
        df.set_index("user_id", drop=False)
        .loc[user_id]
        .set_index("date", drop=False)
        .loc[period]
        .reset_index(drop=True)
    )

## Improve txn data production speed

In [38]:
df = hd.read_raw_sample("777")
hd.inspect(df)

Time for read_raw_sample: 4.02 seconds
(682,656, 27)


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",Debit,25.030001,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,U
1,688291,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,warrington b.c. - d/d,Debit,26.000000,Gym Membership,Gym Membership,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,U


Which module takes the longest

In [102]:
cl.lowercase_categories(df)

Time for lowercase_categories: 1.23 seconds


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",debit,25.030001,no tag,no tag,no tag,no merchant,unknown merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,u
1,688291,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,warrington b.c. - d/d,debit,26.000000,gym membership,gym membership,no tag,no merchant,unknown merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,u
2,688290,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,"9572 24jan12 , apple itunes store, gbp , london gb - pos",debit,6.970000,mobile app,no tag,"entertainment, tv, media",apple,apple,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-08-15,u
3,688292,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,<mdbremoved>,credit,1901.329956,salary or wages (main),salary or wages (main),salary or wages (main),no merchant,no merchant business line,2011-07-20,2020-07-21 20:32:00,2014-07-18,2020-05-19,u
4,688294,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-30,262916,natwest bank,current,364.220001,legal & gen mi c/l - d/d,debit,16.190001,life insurance,no tag,insurance,legal & general,legal & general,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-11-13,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682651,807020573,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,cash barclay <mdbremoved>,debit,20.000000,cash,no tag,cash,no merchant,personal,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
682652,807020571,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,paypal *wetherspooxxxxxxxx001,debit,3.750000,dining or going out,no tag,dining or going out,wetherspoon,wetherspoon,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
682653,807020572,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,amznmktplace amazon.co.uk,debit,28.980000,enjoyment,no tag,enjoyment,amazon,amazon,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
682654,807020569,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,asda store 4519 south elmsall,debit,11.150000,"food, groceries, household",no tag,"food, groceries, household",asda,asda supermarket,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c


In [121]:
k = pd.DataFrame({"col": ["Ho", "ho"]})
k["col"] = k.col.astype("category")
k.col

0    Ho
1    ho
Name: col, dtype: category
Categories (2, object): ['Ho', 'ho']

In [107]:
df.select_dtypes("category").apply(lambda x: x.cat.rename_categories(str.lower))

186 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def f(df):
    cat_vars = df.select_types("category").columns
    df[cat_vars] = df[cat_vars].apply(lambda x: x.cat.re)

In [135]:
clean = df.pipe(md.clean_data)

Time for rename_cols    : 0.03 seconds
Time for clean_headers  : 0.00 seconds
Time for drop_unneeded_vars: 0.02 seconds
Time for add_year_month_variable: 0.04 seconds
Time for drop_first_and_last_month: 0.07 seconds
Time for lowercase_categories: 1.09 seconds
Time for drop_missing_txn_desc: 0.05 seconds
Time for gender_to_female: 0.04 seconds
Time for credit_debit_to_debit: 0.01 seconds
Time for sign_amount    : 0.00 seconds
Time for missings_to_nan: 0.02 seconds
Time for zero_balances_to_missing: 0.00 seconds
Time for add_tag        : 0.41 seconds
Time for tag_corrections: 0.58 seconds
Time for add_tag_group  : 0.15 seconds
Time for drop_type1_dups: 0.30 seconds
Time for order_and_sort : 0.15 seconds
Time for clean_data     : 3.03 seconds


In [336]:
m = md.create_vars(clean)

Time for balances       : 3.68 seconds
Time for income         : 0.35 seconds
Time for age            : 0.07 seconds
Time for entropy_spend_tag_counts: 0.31 seconds
Time for region_name    : 0.59 seconds
Time for create_vars    : 5.04 seconds


In [332]:
m

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,debit,female,latest_balance,merchant_business_line,postcode,tag_auto,tag_manual,tag_up,ym,yob,balance,age,entropy_sptac,region
0,688300,2012-02-01,777,400.000000,<mdbremoved> - s/o,NaN,transfers,NaN,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,364.220001,non merchant mbl,wa1 4,transfers,other account,other account,201202,1969.0,-2192.597168,43.0,2.829955,North West
1,688299,2012-02-01,777,3.030000,aviva pa - d/d,aviva,spend,finance,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,364.220001,aviva,wa1 4,health insurance,life insurance,life insurance,201202,1969.0,-2192.597168,43.0,2.829955,North West
2,688301,2012-02-03,777,8.750000,chart ins log tran - d/d,NaN,NaN,NaN,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,364.220001,NaN,wa1 4,NaN,memberships,memberships,201202,1969.0,-2222.337158,43.0,2.829955,North West
3,688303,2012-02-03,777,0.990000,"9572 02feb12 , apple itunes store, gbp , london gb - pos",apple,spend,services,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,364.220001,apple,wa1 4,"entertainment, tv, media",NaN,mobile app,201202,1969.0,-2222.337158,43.0,2.829955,North West
4,688302,2012-02-03,777,20.000000,"9572 02feb12 , national lottery , inte , watford gb - pos",camelot,spend,services,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,364.220001,camelot,wa1 4,gambling,NaN,gambling,201202,1969.0,-2222.337158,43.0,2.829955,North West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651784,802522114,2020-07-28,587777,-95.000000,<mdbremoved> bgc,NaN,spend,NaN,2020-06-19,2020-06-19,1711781,2020-08-16 18:36:00,barclays,current,False,0.0,5699.700195,NaN,w7 3,refunded purchase,NaN,refunded purchase,202007,1973.0,5703.021484,47.0,2.318646,London
651785,802887369,2020-07-28,587777,5.400000,tfl travel charge tfl.gov.uk/cp,tfl,spend,travel,2020-06-19,2020-06-19,1711784,2020-08-16 20:35:00,american express,credit card,True,0.0,-1844.410034,tfl,w7 3,public transport,NaN,public transport,202007,1973.0,-1815.630371,47.0,2.318646,London
651786,802887368,2020-07-28,587777,10.650000,marks & spencer plc st london,marks and spencer,spend,household,2020-06-19,2020-06-19,1711784,2020-08-16 20:35:00,american express,credit card,True,0.0,-1844.410034,marks & spencer dept store,w7 3,"food, groceries, household",NaN,"food, groceries, household",202007,1973.0,-1815.630371,47.0,2.318646,London
651787,803052582,2020-07-31,587777,3.320000,sanef sa stlxxxxx404 ddr,NaN,NaN,NaN,2020-06-19,2020-06-19,1711781,2020-08-16 18:36:00,barclays,current,True,0.0,5699.700195,NaN,w7 3,NaN,NaN,NaN,202007,1973.0,5699.701172,47.0,2.318646,London


In [239]:
b = clean.pipe(md.create_vars).entropy_sptac

Time for balances       : 3.29 seconds
Time for age            : 0.08 seconds
Time for entropy_spend_tag_counts: 0.36 seconds
Time for region_name    : 0.86 seconds
Time for create_vars    : 4.61 seconds


In [270]:
def f(df):

    cum_spend = (
        df.groupby(["date", "account_id"]).amount.sum().unstack().fillna(0).cumsum()
    )
    g = df.groupby("account_id")
    last_ref_date = g.account_last_refreshed.first().dt.normalize()
    last_ref_balance = g.latest_balance.first()
    return cum_spend, last_ref_date


c, d = f(clean)

In [329]:
a = c.stack().swaplevel()
a.loc[list(k.index)]
# a.loc[(671, '2012-02-01')]

KeyError: "[(152674, '2020-08-16'), (172900, '2020-08-10'), (342833, '2020-08-15'), (378966, '2020-08-04'), (528835, '2020-08-16'), (528846, '2020-08-16'), (528886, '2020-08-16'), (691190, '2020-08-16'), (693652, '2020-08-16'), (771330, '2020-08-16'), (771332, '2020-08-16'), (903296, '2020-08-16'), (915735, '2020-08-16'), (953871, '2020-08-16'), (992247, '2020-08-16'), (1020856, '2020-08-16'), (1076037, '2020-08-16'), (1076038, '2020-08-16'), (1144469, '2020-08-16'), (1156114, '2020-08-16'), (1156116, '2020-08-16'), (1158256, '2020-08-12'), (1165657, '2020-08-16'), (1171063, '2020-08-16'), (1175836, '2020-08-16'), (1297580, '2020-08-16'), (1297581, '2020-08-16'), (1297582, '2020-08-16'), (1297583, '2020-08-16'), (1375596, '2020-08-16'), (1375597, '2020-08-16'), (1375598, '2020-08-16'), (1391830, '2020-08-16'), (1395532, '2020-08-10'), (1396774, '2020-08-16'), (1425165, '2020-08-12'), (1429625, '2020-08-16'), (1429626, '2020-08-16'), (1456982, '2020-08-16'), (1499536, '2020-08-16'), (15